In [103]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
from imblearn.over_sampling import SMOTE

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split, GridSearchCV, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error, r2_score, f1_score, roc_auc_score, classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import SequentialFeatureSelector as SFS, SelectFromModel
from xgboost import XGBClassifier as XGBC
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV
from sklearn.svm import SVC

import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
import copy

In [104]:
merged=pd.read_csv('Mmerged.csv')
mergedT=pd.read_csv('MmergedTest.csv')
mergedH=pd.read_csv('MmergedHoldout.csv')

In [105]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14283 entries, 0 to 14282
Data columns (total 67 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   id                                                               14283 non-null  int64  
 1   hosp_admittime                                                   14283 non-null  object 
 2   hosp_dischtime                                                   14283 non-null  object 
 3   icu_intime                                                       14283 non-null  object 
 4   icu_outtime                                                      14283 non-null  object 
 5   los_icu                                                          14283 non-null  float64
 6   icu_death                                                        14283 non-null  int64  
 7   gender                                  

In [106]:
cla_merged=merged.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_outcome','los_icu'], axis=1)
cla_mergedT=mergedT.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_outcome','los_icu'],axis=1)
cla_mergedH=mergedH.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_outcome','los_icu'],axis=1)

In [107]:
#split X, y

X_train=cla_merged.drop(['icu_death'], axis=1)
y_train=cla_merged['icu_death']

X_test=cla_mergedT.drop(['icu_death'], axis=1)
y_test=cla_mergedT['icu_death']

X_hold=cla_mergedH.drop(['icu_death'], axis=1)
y_hold=cla_mergedH['icu_death']

In [157]:
y_train.value_counts()

0    12756
1     1527
Name: icu_death, dtype: int64

In [108]:
#SMOTE

smote_sampler = SMOTE(random_state=10,sampling_strategy='minority')
X_smote, y_smote = smote_sampler.fit_resample(X_train, y_train)

In [109]:
print(y_train.value_counts())
print(y_smote.value_counts())

X_smote.describe()

0    12756
1     1527
Name: icu_death, dtype: int64
1    12756
0    12756
Name: icu_death, dtype: int64


,gender,admission_age,weight_admit,height,charlson_score,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke,race_encode_African,race_encode_Asian,race_encode_Caucasian,race_encode_Hispanic,race_encode_Not Specified,race_encode_South American,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,hematocrit,hemoglobin,mch,mchc,mcv,platelet,rbc,rdw,wbc,inr,pt,ptt
count,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000,25512.000000
mean,0.471308,68.827674,84.011762,169.520008,6.026458,0.174976,0.088860,0.262112,0.318674,0.074396,0.201748,0.229186,0.580002,0.354304,0.074788,0.078865,0.015640,0.539550,0.021088,0.104147,0.000392,0.025321,0.000078,0.015287,0.000431,0.461704,0.073691,0.039158,0.131546,0.069693,0.085254,0.192145,0.130527,0.003841,0.001646,0.013209,0.092780,0.075102,16.435455,22.000505,36.627673,8.292142,102.992056,2.043778,149.696528,138.108133,4.344523,31.089798,10.113228,30.029938,32.540406,92.115457,194.334521,3.396137,16.023005,12.465789,1.606741,17.372942,38.682388
std,0.499186,14.728104,26.501918,7.725464,2.768441,0.379954,0.284547,0.439792,0.465971,0.262420,0.401313,0.420318,0.493568,0.478311,0.263055,0.269533,0.124079,0.498443,0.143681,0.305457,0.019795,0.157103,0.008854,0.122694,0.020761,0.498541,0.261272,0.193975,0.338003,0.254633,0.279265,0.393994,0.336889,0.061861,0.040542,0.114173,0.290129,0.263561,4.396287,4.759860,24.583324,0.688385,6.308926,1.733962,63.944059,5.001037,0.604264,4.668837,1.545516,2.151457,1.283009,5.880682,96.979801,0.545533,1.985385,7.582076,0.641041,6.787208,13.909575
min,0.000000,18.009528,1.000000,122.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,5.000000,2.000000,4.050000,71.333333,0.100000,32.000000,104.222222,2.300000,13.500000,3.900000,16.700000,25.500000,53.400000,8.000000,1.360000,11.300000,0.100000,0.800000,8.700000,18.250000
25%,0.000000,59.566827,67.744860,166.998080,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.500000,19.000000,19.946505,7.900000,99.107232,1.000000,110.852189,135.500000,3.916667,28.011873,9.100000,28.996875,31.794550,89.000000,13

In [110]:
#normalise
num_cols = X_smote.columns[X_smote.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
X_smote[num_cols] = scaler.fit_transform(X_smote[num_cols])

num_cols = X_test.columns[X_test.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
X_test[num_cols] = scaler.fit_transform(X_test[num_cols])

num_cols = X_hold.columns[X_hold.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
X_hold[num_cols] = scaler.fit_transform(X_hold[num_cols])

# Ridge

In [50]:
l2=RidgeClassifier()
l2.fit(X_smote, y_smote)

RidgeClassifier()

In [51]:
sel = SelectFromModel(l2)
sel.fit(X_smote, y_smote)

selected_feats=sel.get_feature_names_out()
print(selected_feats)
print(len(selected_feats))

['charlson_score' 'ckd' 'race_encode_African' 'race_encode_Caucasian'
 'race_encode_Not Specified' 'admission_type_EW EMER.'
 'admission_type_OBSERVATION ADMIT'
 'admission_type_SURGICAL SAME DAY ADMISSION' 'admission_type_URGENT'
 'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)'
 'first_careunit_Coronary Care Unit (CCU)'
 'first_careunit_Medical Intensive Care Unit (MICU)'
 'first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU)'
 'first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU)'
 'first_careunit_Surgical Intensive Care Unit (SICU)'
 'first_careunit_Trauma SICU (TSICU)' 'aniongap' 'hematocrit' 'mch' 'mchc'
 'mcv']
21


In [52]:
Best_l2 = RidgeClassifierCV(alphas=np.logspace(-6, 6, 13))

Best_l2.fit(X_smote[selected_feats],y_smote)

RidgeClassifierCV(alphas=array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]))

In [53]:
Best_l2.alpha_

10.0

In [54]:
l2_pred = Best_l2.predict(X_test[selected_feats])
# l2_pp=Best_l2.predict_proba(X_test[selected_feats])

# l2_posProb = l2_pp[:,-1]

# rfc_f1 = f1_score(y_test,rfc_pred)
# rfc_auroc = roc_auc_score(y_test, rfc_pp[:,-1], multi_class="ovr")
# print(rfc_f1)
# print(rfc_auroc)
pd.crosstab(y_test, l2_pred)

col_0,0,1
icu_death,,
0,2091,1578
1,114,297


In [55]:
print(classification_report(y_test, l2_pred))

              precision    recall  f1-score   support

           0       0.95      0.57      0.71      3669
           1       0.16      0.72      0.26       411

    accuracy                           0.59      4080
   macro avg       0.55      0.65      0.49      4080
weighted avg       0.87      0.59      0.67      4080



In [56]:
l2_predH = Best_l2.predict(X_hold[selected_feats])
pd.crosstab(y_hold, l2_predH)

col_0,0,1
icu_death,,
0,1043,798
1,58,142


In [57]:
print(classification_report(y_hold, l2_predH))

              precision    recall  f1-score   support

           0       0.95      0.57      0.71      1841
           1       0.15      0.71      0.25       200

    accuracy                           0.58      2041
   macro avg       0.55      0.64      0.48      2041
weighted avg       0.87      0.58      0.66      2041



In [79]:
f1_score(y_hold, l2_predH)

0.24912280701754386

# select from model

In [111]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_smote, y_smote)

sel = SelectFromModel(rfc)
sel.fit(X_smote, y_smote)

SelectFromModel(estimator=RandomForestClassifier(random_state=0))

In [112]:
selected_feats=sel.get_feature_names_out()
print(selected_feats)
print(len(selected_feats))

['admission_age' 'weight_admit' 'height' 'charlson_score'
 'atrial_fibrillation' 'ckd' 'diabetes' 'hypertension' 'ihd'
 'race_encode_Caucasian' 'aniongap' 'bicarbonate' 'bun' 'chloride'
 'creatinine' 'glucose' 'sodium' 'potassium' 'mchc' 'mcv' 'platelet' 'rdw'
 'wbc' 'inr' 'pt' 'ptt']
26


# RF Classifier (select from model)

In [113]:
rfc = RandomForestClassifier(criterion='entropy')

parameters = {'max_depth':np.arange(start=1,stop=10,step=1)}
stratifiedCV = RepeatedStratifiedKFold(n_splits=5)
Best_rfc = GridSearchCV(
    rfc,
    param_grid=parameters,
    cv=stratifiedCV
)

Best_rfc.fit(X_smote[selected_feats],y_smote)

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=5, random_state=None),
             estimator=RandomForestClassifier(criterion='entropy'),
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9])})

In [114]:
Best_rfc.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=9)

In [115]:
rfc_pred = Best_rfc.predict(X_test[selected_feats])

In [116]:
np.count_nonzero(rfc_pred == 3)

0

In [117]:
#Test-RF with forward selected feature

rfc_pred = Best_rfc.predict(X_test[selected_feats])
# rfc_pp=Best_rfc.predict_proba(X_test[selected_feats])

# rfc_posProb = rfc_pp[:,-1]

# # rfc_f1 = f1_score(y_test,rfc_pred)
# rfc_auroc = roc_auc_score(y_test, rfc_pp[:,-1], multi_class="ovr")
# # print(rfc_f1)
# print(rfc_auroc)
pd.crosstab(y_test, rfc_pred)

col_0,0,1
icu_death,,
0,2327,1342
1,95,316


In [118]:
print(classification_report(y_test, rfc_pred))

              precision    recall  f1-score   support

           0       0.96      0.63      0.76      3669
           1       0.19      0.77      0.31       411

    accuracy                           0.65      4080
   macro avg       0.58      0.70      0.53      4080
weighted avg       0.88      0.65      0.72      4080



In [119]:
rfc_predH = Best_rfc.predict(X_hold[selected_feats])
pd.crosstab(y_hold, rfc_predH)

col_0,0,1
icu_death,,
0,1173,668
1,66,134


In [120]:
print(classification_report(y_hold, rfc_predH))

              precision    recall  f1-score   support

           0       0.95      0.64      0.76      1841
           1       0.17      0.67      0.27       200

    accuracy                           0.64      2041
   macro avg       0.56      0.65      0.51      2041
weighted avg       0.87      0.64      0.71      2041



# XGB Classifier

In [121]:
xgbc =XGBC()

xgbc.fit(X_smote,y_smote)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [122]:
thresholds = np.sort(xgbc.feature_importances_)

In [123]:
thresholds

array([0.        , 0.        , 0.00135941, 0.00156619, 0.00169716,
       0.00170006, 0.00173178, 0.00178471, 0.00181485, 0.00184145,
       0.00194124, 0.0019474 , 0.00199871, 0.00204599, 0.0022845 ,
       0.00254125, 0.00255086, 0.00255402, 0.00265304, 0.0027568 ,
       0.00287339, 0.00361757, 0.00363144, 0.00365931, 0.00415735,
       0.00476109, 0.00519479, 0.00591052, 0.00645872, 0.00768044,
       0.0081719 , 0.00848147, 0.00871828, 0.00877368, 0.00950851,
       0.01075724, 0.01530595, 0.0194959 , 0.02122037, 0.02305325,
       0.02723685, 0.02820735, 0.02949702, 0.03000947, 0.03300036,
       0.03369239, 0.03616544, 0.03635908, 0.03802126, 0.03995492,
       0.04497617, 0.04568655, 0.04579935, 0.04891451, 0.0493484 ,
       0.05107951, 0.05201769, 0.05212903, 0.05970402], dtype=float32)

In [124]:
for thresh in thresholds:
    selection = SelectFromModel(xgbc, threshold=thresh, prefit=True)
    select_X_smote = selection.transform(X_smote)
    selection_model = XGBC()
    selection_model.fit(select_X_smote, y_smote)
    select_X_test = selection.transform(X_test)
    predictions = selection_model.predict(select_X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_smote.shape[1], accuracy*100.0))

C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.000, n=59, Accuracy: 74.31%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.000, n=59, Accuracy: 74.31%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


...

C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.049, n=5, Accuracy: 54.95%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.051, n=4, Accuracy: 49.71%
Thresh=0.052, n=3, Accuracy: 32.99%
Thresh=0.052, n=2, Accuracy: 23.95%
Thresh=0.060, n=1, Accuracy: 12.75%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [125]:
selected=pd.DataFrame()
selected['col']=X_smote.columns
selected['rank']=xgbc.feature_importances_
selected.sort_values(by='rank', ascending=False,inplace=True)

top6=selected[:21]
selected_feats=top6['col'].to_list()
selected_feats

['race_encode_Hispanic',
 'race_encode_African',
 'first_careunit_Trauma SICU (TSICU)',
 'atrial_fibrillation',
 'race_encode_Not Specified',
 'admission_type_DIRECT EMER.',
 'hypertension',
 'admission_type_SURGICAL SAME DAY ADMISSION',
 'race_encode_Asian',
 'first_careunit_Surgical Intensive Care Unit (SICU)',
 'ckd',
 'first_careunit_Medical Intensive Care Unit (MICU)',
 'admission_type_OBSERVATION ADMIT',
 'first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU)',
 'ihd',
 'diabetes',
 'race_encode_Caucasian',
 'admission_type_URGENT',
 'first_careunit_Coronary Care Unit (CCU)',
 'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)',
 'cld']

In [126]:

xgb = XGBC()

parameters = {
    'n_estimators':np.arange(start=2,stop=20,step=2),
    'max_depth':np.arange(start=2,stop=6,step=1),
    'learning_rate':np.arange(start=0.05,stop=0.4,step=0.05)
}
stratifiedCV = RepeatedStratifiedKFold(n_splits=5)
Best_xgb = GridSearchCV(
    xgb,
    param_grid=parameters,
    cv=stratifiedCV,
    verbose=1,
    n_jobs=-1
)

Best_xgb.fit(X_smote[selected_feats],y_smote)

Fitting 50 folds for each of 252 candidates, totalling 12600 fits


GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=5, random_state=None),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     impo...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35]),
                         'max_depth': array([2, 3, 4, 5]),
                         'n_estimators': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])},
             verbose=1)

In [127]:
Best_xgb.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.35000000000000003,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=18, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [128]:

xgb_pred = Best_xgb.predict(X_test[selected_feats])
# xgb_pp=Best_xgb.predict_proba(X_test[selected_feats])

# xgb_posProb = xgb_pp[:,-1]

# xgb_f1 = f1_score(y_test, xgb_pred)
# xgb_auroc = roc_auc_score(y_test, xgb_pp[:,-1])
# print(xgb_f1)
# print(xgb_auroc)
pd.crosstab(y_test, xgb_pred)

col_0,0,1
icu_death,,
0,3520,149
1,368,43


In [129]:
print(classification_report(y_test, xgb_pred))

              precision    recall  f1-score   support

           0       0.91      0.96      0.93      3669
           1       0.22      0.10      0.14       411

    accuracy                           0.87      4080
   macro avg       0.56      0.53      0.54      4080
weighted avg       0.84      0.87      0.85      4080



In [130]:
xgb_predH = Best_xgb.predict(X_hold[selected_feats])
pd.crosstab(y_hold, xgb_predH)

col_0,0,1
icu_death,,
0,1763,78
1,185,15


In [131]:
print(classification_report(y_hold, xgb_predH))

              precision    recall  f1-score   support

           0       0.91      0.96      0.93      1841
           1       0.16      0.07      0.10       200

    accuracy                           0.87      2041
   macro avg       0.53      0.52      0.52      2041
weighted avg       0.83      0.87      0.85      2041



# Support vector

In [145]:
names=X_smote.columns.to_numpy()
names

array(['gender', 'admission_age', 'weight_admit', 'height',
       'charlson_score', 'atrial_fibrillation', 'malignant_cancer', 'chf',
       'ckd', 'cld', 'copd', 'diabetes', 'hypertension', 'ihd', 'stroke',
       'race_encode_African', 'race_encode_Asian',
       'race_encode_Caucasian', 'race_encode_Hispanic',
       'race_encode_Not Specified', 'race_encode_South American',
       'admission_type_DIRECT EMER.', 'admission_type_DIRECT OBSERVATION',
       'admission_type_ELECTIVE', 'admission_type_EU OBSERVATION',
       'admission_type_EW EMER.', 'admission_type_OBSERVATION ADMIT',
       'admission_type_SURGICAL SAME DAY ADMISSION',
       'admission_type_URGENT',
       'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)',
       'first_careunit_Coronary Care Unit (CCU)',
       'first_careunit_Medical Intensive Care Unit (MICU)',
       'first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU)',
       'first_careunit_Neuro Intermediate',
       'first_careunit_

In [146]:
sv = SVC(kernel='linear')
sv.fit(X_smote, y_smote)

coef =abs(sv.coef_[0])
coef

array([5.60423542e-06, 1.04039597e-05, 2.73397006e-06, 4.19268760e-06,
       1.08599643e-05, 3.23396619e-06, 8.54378465e-06, 6.00411760e-06,
       1.80504606e-05, 5.39355965e-06, 3.99129854e-06, 1.17494985e-05,
       1.44013261e-05, 4.51344430e-06, 3.96500642e-06, 5.39085743e-01,
       2.48173955e-01, 9.96920603e-01, 2.87379050e-01, 6.10940766e-01,
       3.95954557e-02, 3.14217197e-01, 1.77076217e-02, 2.45394479e-01,
       4.15284004e-02, 9.97117073e-01, 5.22571110e-01, 3.87973166e-01,
       6.76028754e-01, 5.09283452e-01, 5.58552126e-01, 7.88024014e-01,
       6.73816014e-01, 1.23740687e-01, 8.10934925e-02, 2.28358135e-01,
       5.80287818e-01, 5.27151130e-01, 2.75060188e-05, 1.49977193e-05,
       7.71018608e-06, 4.10943161e-06, 9.24958796e-06, 7.93244790e-06,
       1.93670928e-07, 1.52209303e-06, 7.69167199e-08, 5.53578404e-06,
       8.75494124e-06, 2.24318109e-04, 1.02530532e-04, 1.95899401e-04,
       1.82982078e-06, 2.08896728e-05, 2.87972486e-06, 2.85049957e-07,
      

In [147]:
sv_rank = pd.DataFrame({'features': names, 'coef': coef})
sv_rank.sort_values(by=['coef'], inplace=True, ascending=False)
sv_rank

,features,coef
25,admission_type_EW EMER.,9.971171e-01
17,race_encode_Caucasian,9.969206e-01
31,first_careunit_Medical Intensive Care Unit (MICU),7.880240e-01
28,admission_type_URGENT,6.760288e-01
32,first_careunit_Medical/Surgical Intensive Care...,6.738160e-01
19,race_encode_Not Specified,6.109408e-01
36,first_careunit_Surgical Intensive Care Unit (S...,5.802878e-01
30,first_careunit_Coronary Care Unit (CCU),5.585521e-01
15,race_encode_African,5.390857e-01
37,first_careunit_Trauma SICU (TSICU),5.271511e-01


In [148]:
top=sv_rank[:12]
selected_feats=top['features'].to_list()
selected_feats

['admission_type_EW EMER.',
 'race_encode_Caucasian',
 'first_careunit_Medical Intensive Care Unit (MICU)',
 'admission_type_URGENT',
 'first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU)',
 'race_encode_Not Specified',
 'first_careunit_Surgical Intensive Care Unit (SICU)',
 'first_careunit_Coronary Care Unit (CCU)',
 'race_encode_African',
 'first_careunit_Trauma SICU (TSICU)',
 'admission_type_OBSERVATION ADMIT',
 'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)']

In [149]:
sv = SVC(kernel='rbf')

parameters = {'C':np.arange(0.5,5,step=0.5)}
stratifiedCV = RepeatedStratifiedKFold(n_splits=5)
sv_f0 = GridSearchCV(
    sv,
    param_grid=parameters,
    scoring='f1',
    cv=stratifiedCV,
    verbose=1,
    n_jobs=-1
)

sv_f0.fit(X_smote[selected_feats],y_smote)

Fitting 50 folds for each of 9 candidates, totalling 450 fits


GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=5, random_state=None),
             estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5])},
             scoring='f1', verbose=1)

In [150]:
sv_f0.best_estimator_

SVC(C=4.0)

In [151]:
sv_f0.best_score_

0.8541935163102292

In [152]:
sv_f0_pred = sv_f0.predict(X_test[selected_feats])
pd.crosstab(y_test, sv_f0_pred)

col_0,0,1
icu_death,,
0,3356,313
1,369,42


In [153]:
print(classification_report(y_test, sv_f0_pred))

              precision    recall  f1-score   support

           0       0.90      0.91      0.91      3669
           1       0.12      0.10      0.11       411

    accuracy                           0.83      4080
   macro avg       0.51      0.51      0.51      4080
weighted avg       0.82      0.83      0.83      4080



In [154]:
sv_f0_predH = sv_f0.predict(X_hold[selected_feats])
pd.crosstab(y_hold, sv_f0_predH)

col_0,0,1
icu_death,,
0,1672,169
1,170,30


In [155]:
print(classification_report(y_hold, sv_f0_predH))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      1841
           1       0.15      0.15      0.15       200

    accuracy                           0.83      2041
   macro avg       0.53      0.53      0.53      2041
weighted avg       0.83      0.83      0.83      2041

